In [2]:
import numpy as np 
import seaborn as sns
import pandas as pd

In [4]:
df=pd.read_csv('~/Downloads/train.csv')

In [7]:
english= df[df.language=='English']

In [19]:
train=english[['premise', 'hypothesis', 'label']]

In [13]:
from bert_serving.client import BertClient

In [14]:
bc = BertClient()

In [16]:
embedding = bc.encode(["I love data science and analytics vidhya."])
# check the shape of embedding, it should be 1x768
print(embedding.shape)


(1, 768)


In [20]:
import re

In [32]:
# clean text from noise
def clean_text(text):
    # filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [31]:
train.shape

(6870, 4)

In [36]:
train.clean_premise= [clean_text(x) for x in train.loc[:, ('premise')]]

In [46]:
train['clean_hypothesis']=[clean_text(x) for x in train.loc[:, ('hypothesis')]]

<ipython-input-46-8ab69d8a1f6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clean_hypothesis']=[clean_text(x) for x in train.loc[:, ('hypothesis')]]


In [47]:
train

,premise,hypothesis,label,clean_premise,clean_hypothesis
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0,and these comments were considered in formulat...,the rules developed in the interim were put to...
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2,these are issues that we wrestle with in pract...,practice groups are not permitted to work on t...
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0,you know they can't really defend themselves l...,they can't defend themselves because of their ...
7,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,2,from cockpit country to st ann's bay,from st ann's bay to cockpit country
8,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,1,look it's your skin but you're going to be i...,the boss will fire you if he sees you slacking...
...,...,...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2,the results of even the most well designed epi...,all studies have the same amount of uncertaint...
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0,but there are two kinds of the pleasure of do...,but there are two kinds of the pleasure of doi...
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2,the important thing is to realize that it's wa...,it cannot be moved now or ever
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2,at the west end is a detailed model of the who...,the model temple complex is at the east end


In [49]:
from sklearn.model_selection import train_test_split

In [50]:
X_tr, X_val, y_tr, y_val = train_test_split(train[['clean_premise', 'clean_hypothesis']], train.label, test_size=.25, random_state=0)

In [52]:
from bert_serving.client import BertClient

# make a connection with the BERT server using it's ip address
bc = BertClient()
# get the embedding for train and val sets
X_tr_bert = bc.encode(X_tr)
X_val_bert = bc.encode(X_val)

TypeError: "                                           clean_premise  \
5538   be sure to look around and compare before buying    
4267   our efforts having been in vain  we had abando...   
12001  they said that     agencies need to be able to...   
10938    the wagon man got killed when they attacked him   
1844   of how  when tea was done  and everyone had st...   
...                                                  ...   
8748   there are other reasons that wrecks cause fan ...   
5762   uh whether one might conceive  no pun intended...   
2957   what's more  there is no evidence of any compe...   
4620   more to the point  even as the major airlines ...   
4830   but japan was reluctant to sue for peace becau...   

                                        clean_hypothesis  
5538   make sure to browse and make comparisons befor...  
4267      we were no longer trying to solve the problem   
12001  it was stated that each entity should match th...  
10938               the wagon m

### USE CLS